<a href="https://colab.research.google.com/github/borgesjose/reconhecimento_de_padroes/blob/master/WEEK%206/pearson_correlation_atv_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install scikit-optimize

In [ ]:
pip install xgboost

# Atividade 6 -  Seleção de features

Atividade (1.25): realizar a seleção de de características baseada no coeficiente de correlação de Pearson atendendo aos seguintes itens:
<ol>
        <li> Criar uma função chamada feature_selector que irá receber um limiar como parâmetro de entrada e deverá retornar um subconjunto dos dados referente aquelas features cuja correlação está acima do limiar passado; </li> 
        <li> Extrapolar a função get_best_model para abranger os classificadores svm e gxboost;</li> 
        <li> Extrapolar a função get_best_model para abranger a otimização Bayesiana (Scikit-Optimize);</li> 
        <li> Apresentar uma tabela de comparação dos resultado da classificação considerando cada 1 dos otimizadores: acurácia de teste, kappa de teste e tempo médio do otimizador para fitar os dados. Dica: divisão treino é a sua escolha</li>    
</ol 

### Importando as bibliotecas

In [ ]:
import pandas as pd
from scipy import stats

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC 

import xgboost as xgb

### Carregandos Dados

In [ ]:
# Fonte:  https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients
#data = pd.read_csv('credit-card-default.csv')
data = pd.read_csv('https://raw.githubusercontent.com/borgesjose/reconhecimento_de_padroes/master/WEEK%206/credit-card-default.csv')

## 1. Criando a função ``` feature_selector``` :

In [ ]:
def feature_selector(lim,y_name):

    x = data.drop(y_name, axis=1)
    
    highly_corr_feat = data.columns[data.corr()[y_name].abs() > lim]
    highly_corr_feat = highly_corr_feat.drop(y_name)

    x_corr = x[highly_corr_feat]

    return x_corr

In [ ]:
feature_selector(0.2,'default payment next month' )

,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5
0,2,2,-1,-1,-2
1,-1,2,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,-1,0,-1,0,0
...,...,...,...,...,...
29995,0,0,0,0,0
29996,-1,-1,-1,-1,0
29997,4,3,2,-1,0
29998,1,-1,0,0,0


## 2. Extrapolando a função ```get_best_model``` para abranger os classificadores svm e gxboost :

In [ ]:
def get_best_model_1(model_name, x, y):
    
    if model_name == 'd_tree':
        params = {'max_depth': [None, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]}
        model = DecisionTreeClassifier()
    elif model_name == 'r_forest':
        params = {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 1, 3, 5, 7]}
        model = RandomForestClassifier()
    elif model_name == 'svm':
        params = {'C': [1, 10, 100, 1000]}#, 'gamma': [0.001, 0.0001]}
        model = SVC()
    elif model_name == 'gxboost':
        params = {'max_depth': [2, 4, 6], 'n_estimators': [50, 100, 200]}
        model = model = xgb.XGBClassifier()
        
    else:
        print("Oops! That was no valid model. Try again...")
    
    
    grid = GridSearchCV(model,           
                        params,          
                        error_score=0.)  
    grid.fit(x, y) 
    
    print("Maior acurácia: {}".format(grid.best_score_))
    print("Melhores parâmetros: {}".format(grid.best_params_))
    print("Tempo médio para o otimizador fitar os dados (s): {}".format(round(grid.cv_results_['mean_fit_time'].mean(), 3)))
    print("Tempo médio para o otimizador realizar a predição (s): {}".format(round(grid.cv_results_['mean_score_time'].mean(), 3)))
    
    results = np.array([grid.best_score_ ,grid.best_params_ ,round(grid.cv_results_['mean_fit_time'].mean(), 3), round(grid.cv_results_['mean_score_time'].mean(), 3)])
    
    
    return pd.Series(data=results, index=['acurácia', 'Melhores parâmetros', 'Kappa','Time opt fit', 'Time opt pred'])



## 3. Extrpolando a função ```get_best_model``` para abranger a otimização Bayesiana (Scikit-Optimize):

In [ ]:
# Bayes Search Scikit-learn
from skopt import BayesSearchCV
import numpy as np

from sklearn.metrics import cohen_kappa_score

from sklearn.model_selection import train_test_split

import xgboost as xgb

In [ ]:
def get_best_model_2(model_name, opt_name, x, y):
    
    import warnings
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3,random_state=109) # 70% training and 30% test

    
    if model_name == 'd_tree':
        params = {'max_depth': [None, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]}
        model = DecisionTreeClassifier()
    elif model_name == 'r_forest':
        params = {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 1, 3, 5, 7]}
        model = RandomForestClassifier()
    elif model_name == 'svm':
        params = {'C': [1, 10, 50, 100]} #, 'gamma': [0.001, 0.0001]}
        model = SVC()
    elif model_name == 'gxboost':
        params = {'max_depth': [2, 4, 6], 'n_estimators': [50, 100, 200]}
        model = xgb.XGBClassifier()
    else:
        print("Oops! That was no valid model. Try again...")
    
    if opt_name == 'grid':
        grid = GridSearchCV(model,           
                            params,          
                            error_score=0.)  
        grid.fit(X_train, y_train) 

        kappa = cohen_kappa_score(y_train, grid.predict(X_train))

    elif opt_name == 'bayes':

        warnings.filterwarnings("ignore")

        grid = BayesSearchCV(model, params)
        grid.fit(X_train,y_train)

        kappa = cohen_kappa_score(y_train, grid.predict(X_train))

    else:
        print("Oops! That was no valid optmization method. Try again...")
    
    print("Maior acurácia: {}".format(grid.best_score_))
    print("Melhores parâmetros: {}".format(grid.best_params_))
    print("Cohen Kappa: {}".format(kappa))
    print("Tempo médio para o otimizador fitar os dados (s): {}".format(round(grid.cv_results_['mean_fit_time'].mean(), 3)))
    print("Tempo médio para o otimizador realizar a predição (s): {}".format(round(grid.cv_results_['mean_score_time'].mean(), 3)))

    results = np.array([grid.best_score_ ,grid.best_params_ , kappa ,round(grid.cv_results_['mean_fit_time'].mean(), 3), round(grid.cv_results_['mean_score_time'].mean(), 3)])
    
    
    return pd.Series(data=results, index=['acurácia', 'Melhores parâmetros', 'Kappa','Time opt fit', 'Time opt pred'])

In [ ]:
#gx_test = get_best_model_2('gxboost', "grid",x_corr, y)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(x_corr, y, test_size=0.3,random_state=109) # 70% training and 30% test

#params = {'max_depth': [None, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]}
#model = DecisionTreeClassifier()

#grid = BayesSearchCV(model, params)
#grid.fit(X_train,y_train)

#cohen_kappa_score(y_train, grid.predict(X_train))

#print(grid.predict(X_train))

## 4. Apresentando resultados:

Apresentar uma tabela de comparação dos resultado da classificação considerando cada 1 dos otimizadores: acurácia de teste, kappa de teste e tempo médio do otimizador para fitar os dados. Dica: divisão treino é a sua escolha

In [ ]:
y = data['default payment next month']
x_corr = feature_selector(0.2,'default payment next month' )

In [ ]:
bayes_table_d_tree = get_best_model_2('d_tree', "bayes",x_corr, y)

Maior acurácia: 0.8210000000000001
Melhores parâmetros: OrderedDict([('max_depth', 3)])
Cohen Kappa: 0.38841240348306516
Tempo médio para o otimizador fitar os dados (s): 0.014
Tempo médio para o otimizador realizar a predição (s): 0.002


In [ ]:
bayes_table_d_tree = get_best_model_2('d_tree', "bayes",x_corr, y)
bayes_table_r_forest = get_best_model_2('r_forest', "bayes",x_corr, y)
bayes_table_svm = get_best_model_2('svm', "bayes",x_corr, y)
bayes_table_gx = get_best_model_2('gxboost', "bayes",x_corr, y)

bayes_table = pd.concat([bayes_table_d_tree, bayes_table_r_forest,bayes_table_svm,bayes_table_gx], axis=1)
bayes_table.columns = ['d_tree','r_forest','svm','gxboost']

Maior acurácia: 0.8210000000000001
Melhores parâmetros: OrderedDict([('max_depth', 3)])
Cohen Kappa: 0.38841240348306516
Tempo médio para o otimizador fitar os dados (s): 0.013
Tempo médio para o otimizador realizar a predição (s): 0.002
Maior acurácia: 0.8216666666666667
Melhores parâmetros: OrderedDict([('max_depth', 3), ('n_estimators', 50)])
Cohen Kappa: 0.3770752201594626
Tempo médio para o otimizador fitar os dados (s): 0.49
Tempo médio para o otimizador realizar a predição (s): 0.037
Maior acurácia: 0.8200000000000001
Melhores parâmetros: OrderedDict([('C', 1)])
Cohen Kappa: 0.38735530523563244
Tempo médio para o otimizador fitar os dados (s): 10.204
Tempo médio para o otimizador realizar a predição (s): 0.739
Maior acurácia: 0.8215238095238094
Melhores parâmetros: OrderedDict([('max_depth', 2), ('n_estimators', 50)])
Cohen Kappa: 0.3798942777500248
Tempo médio para o otimizador fitar os dados (s): 0.679
Tempo médio para o otimizador realizar a predição (s): 0.014


In [ ]:
bayes_table = pd.concat([bayes_table_d_tree, bayes_table_r_forest,bayes_table_svm,bayes_table_gx], axis=1)
bayes_table.columns = ['d_tree','r_forest','svm','gxboost']

In [ ]:
bayes_table 

,d_tree,r_forest,svm,gxboost
acurácia,0.821,0.821667,0.82,0.821524
Melhores parâmetros,{'max_depth': 3},"{'max_depth': 3, 'n_estimators': 50}",{'C': 1},"{'max_depth': 2, 'n_estimators': 50}"
Kappa,0.388412,0.377075,0.387355,0.379894
Time opt fit,0.013,0.49,10.204,0.679
Time opt pred,0.002,0.037,0.739,0.014


In [ ]:
grid_table_d_tree = get_best_model_2('d_tree', "grid",x_corr, y)
grid_table_r_forest = get_best_model_2('r_forest', "grid",x_corr, y)
grid_table_svm = get_best_model_2('svm', "grid",x_corr, y)
grid_table_gx = get_best_model_2('gxboost', "grid",x_corr, y)

grid_table = pd.concat([grid_table_d_tree, grid_table_r_forest,grid_table_svm,grid_table_gx], axis=1)
grid_table.columns = ['d_tree','r_forest','svm','gxboost']

Maior acurácia: 0.8210000000000001
Melhores parâmetros: {'max_depth': 3}
Cohen Kappa: 0.38841240348306516
Tempo médio para o otimizador fitar os dados (s): 0.011
Tempo médio para o otimizador realizar a predição (s): 0.002
Maior acurácia: 0.8218095238095238
Melhores parâmetros: {'max_depth': 3, 'n_estimators': 200}
Cohen Kappa: 0.38013271101294943
Tempo médio para o otimizador fitar os dados (s): 0.446
Tempo médio para o otimizador realizar a predição (s): 0.035
Maior acurácia: 0.8200000000000001
Melhores parâmetros: {'C': 1}
Cohen Kappa: 0.38735530523563244
Tempo médio para o otimizador fitar os dados (s): 10.861
Tempo médio para o otimizador realizar a predição (s): 0.744
Maior acurácia: 0.8215238095238094
Melhores parâmetros: {'max_depth': 2, 'n_estimators': 50}
Cohen Kappa: 0.3798942777500248
Tempo médio para o otimizador fitar os dados (s): 0.772
Tempo médio para o otimizador realizar a predição (s): 0.016


In [ ]:

grid_table = pd.concat([grid_table_d_tree, grid_table_r_forest,grid_table_svm,grid_table_gx], axis=1)
grid_table.columns = ['d_tree','r_forest','svm','gxboost']

In [ ]:
grid_table

,d_tree,r_forest,svm,gxboost
acurácia,0.821,0.82181,0.82,0.821524
Melhores parâmetros,{'max_depth': 3},"{'max_depth': 3, 'n_estimators': 200}",{'C': 1},"{'max_depth': 2, 'n_estimators': 50}"
Kappa,0.388412,0.380133,0.387355,0.379894
Time opt fit,0.011,0.446,10.861,0.772
Time opt pred,0.002,0.035,0.744,0.016


## Conclusão

Observando o 